In [5]:
import os
import numpy as np
import random
import math
from collections import Counter

In [6]:
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]
    all_words = []
    for mail in emails:
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:  #Body of email is only 3rd line of text file
                    words = line.split()
                    all_words += words
    dictionary = Counter(all_words)
    dictionary_temp = Counter(all_words)

    # Paste code for non-word removal here(code snippet is given below)
    list_to_remove = dictionary_temp.keys()
    for item in list_to_remove:
        if item.isalpha() == False:    #Determine whether it is punctuation
            del dictionary[item]
        elif len(item) == 1:           #
            del dictionary[item]
    dictionary = dictionary.most_common(3000)
    return dictionary

In [7]:
def extract_features(root_dir,dictionary):
    emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
    all_words = []
    features_matrix = np.zeros((len(emails),3000))
    docID = 0
    for mail in emails:
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:
                    words = line.split()
                    for word in words:
                        wordID = 0
                        for i,d in enumerate(dictionary):
                            if d[0] == word:
                                wordID = i
                                features_matrix[docID,wordID] = words.count(word)
        docID = docID + 1
    return features_matrix
    

In [9]:
Path = "train-mails"
dir = make_Dictionary(Path)
# print('finish dir:\n',dir)
train_matrix = extract_features(Path,dir)
print(train_matrix)

Path_test = "test-mails"
test_matrix = extract_features(Path_test,dir)
print(test_matrix)

', 16), ('beyond', 16), ('prosodic', 16), ('complement', 16), ('shall', 16), ('verbal', 16), ('representative', 16), ('round', 16), ('challenge', 16), ('aware', 16), ('monitor', 16), ('latin', 16), ('bob', 16), ('responsibility', 16), ('reasonable', 16), ('literally', 16), ('jan', 16), ('ren', 16), ('character', 16), ('eventually', 16), ('light', 16), ('approximately', 16), ('parameter', 16), ('preposition', 16), ('ease', 16), ('examine', 16), ('attitude', 16), ('equivalent', 16), ('diurnal', 16), ('humanity', 16), ('pencil', 16), ('phillip', 16), ('tend', 16), ('italy', 16), ('possibly', 16), ('scandinavian', 16), ('reap', 16), ('christmas', 16), ('village', 16), ('inside', 16), ('appeal', 16), ('successfully', 16), ('statistics', 16), ('wall', 16), ('coupon', 16), ('browser', 16), ('smtp', 16), ('specialize', 16), ('extraordinary', 16), ('vision', 16), ('gay', 16), ('liar', 16), ('boss', 16), ('racer', 16), ('snail', 16), ('tips', 16), ('reg', 16), ('golden', 16), ('secrets', 16), ('

In [63]:
class NaiveBayes:
    train_data = []
    Mean = {}
    Std = {}
    Prob = {}
    def fit(self,x,y):
        #divide x by x
        DivideX = {}
        for i in range(len(y)):
            if (y[i] not in DivideX):
                DivideX[y[i]]=[]
                DivideX[y[i]].append(x[i])
            else:
                DivideX[y[i]] = np.vstack((DivideX[y[i]],x[i]))
        self.train_data = DivideX
        for key in DivideX:
            self.Mean[key] = []
            self.Std[key] = []
            self.Prob[key] = []
            for i in range(DivideX[key].shape[1]):
                self.Mean[key].append(np.mean(DivideX[key][:,i]))
                #Divide
                c = 0
                for j in DivideX[key][:,i]:
                    if j >= self.Mean[key][i]:
                        c += 1
                num = len(DivideX[key][:,i])
                self.Prob[key].append(c + 1/num + 1)
                #
                if np.std(DivideX[key][:,i]) == 0:
                    self.Std[key].append(0.00001)
                else:
                    self.Std[key].append(np.std(DivideX[key][:,i]))
        # print(self.Mean)
    def probabilty(self,data,key):
        prob = 1
        # print(data)
        for i in range(len(data)):
            if self.Std[key][i] == 0:
                if data[i] >= self.Mean[key][i]:
                    print('!!!')
                    prob *= self.Prob[key][i]
                else:
                    prob *= (1 - self.Prob[key][i])
            else:
                exp = math.exp(-(math.pow(data[i] - self.Mean[key][i],2)/(2*math.pow(self.Std[key][i],2))))
                prob *= (1/(math.sqrt(2*math.pi)*self.Std[key][i])) * exp
        return prob
    def predict(self,data):
        result = []
        for i in range(data.shape[0]):
            MaxProb = 0
            MaxProb_key = 0
            for key in self.Mean:
                Prob_temp = self.probabilty(data[i],key)
                if(Prob_temp > MaxProb):
                    MaxProb = Prob_temp
                    MaxProb_key = key
            result.append(MaxProb_key)
        return result
    def accuracy(self,data,prediction):
        correct = 0
        for x in range(len(data)):
            if data[x] == prediction[x]:
                correct += 1
        return (correct/float(len(data))) * 100


In [70]:
NB = NaiveBayes()
train_labels = np.zeros(train_matrix.shape[0])
train_labels[351:701] = 1
NB.fit(train_matrix,train_labels)
pred = NB.predict(train_matrix)
print(NB.accuracy(train_labels,pred))

89.88603988603988


In [71]:
pred_test = NB.predict(test_matrix)
test_label = np.zeros(test_matrix.shape[0])
test_label[130:] = 1
# print(pred_test)
print(NB.accuracy(test_label,pred_test))

80.38461538461539


In [72]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
GNB = GaussianNB()
GNB.fit(train_matrix,train_labels)
y_pred = GNB.predict(train_matrix)
print(NB.accuracy(train_labels,y_pred))
y_pred_test = GNB.predict(test_matrix)
print(NB.accuracy(test_label,y_pred_test))

98.14814814814815
95.76923076923077
